In [ ]:
# %load open3dVis.py
import open3d as o3d
import numpy as np
from matplotlib import pyplot as plt


# Define Plane Segmentation Function
def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud],
                                      zoom=0.1,
                                      front=[0.02, 0.85, 0.52],
                                      lookat=[1.61, 0.27, 0.30],
                                      up=[-0.05, -0.52, 0.85],
                                      window_name='Open3D Removal Outlier')


# Load a pointcloud
pcd = o3d.io.read_point_cloud("pointcloud1.pcd")
print(pcd)
print(np.asarray(pcd.points))
# Visualize the original pcd doc from a specific perspective
o3d.visualization.draw_geometries([pcd],
                                  zoom=0.1,
                                  front=[0.02, 0.85, 0.52],
                                  lookat=[1.61, 0.27, 0.30],
                                  up=[-0.05, -0.52, 0.85])


# Down-sampling：Down-sample the point cloud with a voxel of 0.1
downpcd = pcd.voxel_down_sample(voxel_size=0.1)
print(downpcd)
o3d.visualization.draw_geometries([downpcd],
                                  zoom=0.1,
                                  front=[0.02, 0.85, 0.52],
                                  lookat=[1.61, 0.27, 0.30],
                                  up=[-0.05, -0.52, 0.85],
                                  window_name='Open3D downSample')


# Statistical outlier removal
cl, ind = downpcd.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
display_inlier_outlier(downpcd, ind)
# ‘downpcd_inlier_cloud’ indicates the part of the points to be selected
downpcd_inlier_cloud = downpcd.select_by_index(ind)
print(downpcd_inlier_cloud)


# Plane Segmentation
plane_model, inliers = downpcd_inlier_cloud.segment_plane(distance_threshold=0.2,
                                                          ransac_n=10,
                                                          num_iterations=10000)
[a, b, c, d] = plane_model
# Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0
# Define inlier_cloud and outlier_cloud
inlier_cloud = downpcd_inlier_cloud.select_by_index(inliers)
outlier_cloud = downpcd_inlier_cloud.select_by_index(inliers, invert=True)


# DBscan
with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(outlier_cloud.cluster_dbscan(eps=1.5, min_points=10, print_progress=True))
# Find the number of clusters of the pointcloud
max_label = labels.max()
# Visualization
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
outlier_cloud.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([outlier_cloud],
                                  zoom=0.1,
                                  front=[0.02, 0.85, 0.52],
                                  lookat=[1.61, 0.27, 0.30],
                                  up=[-0.05, -0.52, 0.85],
                                  window_name='Open3D Plane Model')


# Finally add the bounding box
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(outlier_cloud)
# Adding bounding boxes to different clusters
for i in range(0, max_label + 1):
    ind = np.where(labels == i)[0]
    cluster_cloud = outlier_cloud.select_by_index(ind)
    aabb = cluster_cloud.get_axis_aligned_bounding_box()
    aabb.color = (1, 0, 0)
    vis.add_geometry(aabb)
vis.run()
